# MODELO DEEP LEARNING

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/finalProyect')

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Ruta del archivo procesado en Google Drive
file_path_processed = "/content/drive/MyDrive/finalProyect/dataset_procesado.csv"

# Cargar el conjunto de datos procesado
df_processed = pd.read_csv(file_path_processed)

# Verificar las columnas disponibles en el conjunto de datos
print("Columnas disponibles en el conjunto de datos:")
print(df_processed.columns)

# Dividir el conjunto de datos en características (X) y variable objetivo (y)
X = df_processed.drop("income", axis=1)
y = df_processed["income"]

# Convertir las etiquetas categóricas a valores numéricos
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Definir las columnas categóricas para One-Hot Encoding
categorical_cols = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']

# Crear un transformador para aplicar One-Hot Encoding a las columnas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse=False), categorical_cols)
    ],
    remainder='passthrough'
)

# Crear un pipeline con el preprocesador y el escalador
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler(with_mean=False))  # Ajustar el parámetro with_mean
])

# Aplicar el pipeline al conjunto de entrenamiento y prueba
X_train_scaled = pipeline.fit_transform(X_train)
X_test_scaled = pipeline.transform(X_test)


# Crear modelo de red neuronal
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train_scaled, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test_encoded))

# Realizar predicciones en el conjunto de prueba
y_probabilities = model.predict(X_test_scaled)

# Obtener las clases predichas (aplicando un umbral de 0.5)
y_pred_dl = (y_probabilities > 0.5).astype(int)

# Convertir las predicciones a etiquetas
y_pred_dl = y_pred_dl.flatten()

# Convertir las etiquetas codificadas de vuelta a las etiquetas originales
y_test_original = label_encoder.inverse_transform(y_test_encoded)
y_pred_original = label_encoder.inverse_transform(y_pred_dl)

# Evaluar el rendimiento del modelo
accuracy_dl = accuracy_score(y_test_encoded, y_pred_dl)
classification_rep_dl = classification_report(y_test_encoded, y_pred_dl)
conf_matrix_dl = confusion_matrix(y_test_encoded, y_pred_dl)

# Mostrar resultados del modelo de Deep Learning
print(f"Accuracy del modelo de Deep Learning: {accuracy_dl}")
print("\nClassification Report del modelo de Deep Learning:\n", classification_rep_dl)
print("\nConfusion Matrix del modelo de Deep Learning:\n", conf_matrix_dl)

Columnas disponibles en el conjunto de datos:
Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income', 'capital_ratio', 'age_scaled'],
      dtype='object')
Epoch 1/10
814/814 [==============================] - 5s 4ms/step - loss: 0.3596 - accuracy: 0.8320 - val_loss: 0.3261 - val_accuracy: 0.8503
Epoch 2/10
814/814 [==============================] - 2s 3ms/step - loss: 0.3204 - accuracy: 0.8489 - val_loss: 0.3287 - val_accuracy: 0.8462
Epoch 3/10
814/814 [==============================] - 2s 3ms/step - loss: 0.3142 - accuracy: 0.8529 - val_loss: 0.3255 - val_accuracy: 0.8540
Epoch 4/10
814/814 [==============================] - 2s 3ms/step - loss: 0.3091 - accuracy: 0.8553 - val_loss: 0.3243 - val_accuracy: 0.8528
Epoch 5/10
814/814 [==============================] - 3s 4ms/step - loss: 0.3047 - accuracy: 0.8575 - v

# AJUSTE DE HIPERPARAMETROS con NEPTUNE.AI

In [ ]:
!pip install --upgrade neptune-client

In [ ]:
!pip install neptune-client

In [ ]:
!pip install neptune-contrib

In [ ]:
!pip install neptune-tensorflow-keras

In [ ]:
import neptune.new as neptune
from neptune.new.integrations.tensorflow_keras import NeptuneCallback
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Iniciar la ejecución en Neptune
run = neptune.init(
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxYzlmZTUwMy02NGJjLTQ2NGYtOWE1Yi03MTlmNTBmMjgzMzUifQ==",
    project="frubengarcia/final",
    name="Your Run Name",  # Puedes cambiar el nombre de la ejecución según tus necesidades
    params={"learning_rate": 0.001, "optimizer": "Adam"}
)

# Ruta del archivo procesado en Google Drive
file_path_processed = "/content/drive/MyDrive/finalProyect/dataset_procesado.csv"

# Cargar el conjunto de datos procesado
df_processed = pd.read_csv(file_path_processed)

# Dividir el conjunto de datos en características (X) y variable objetivo (y)
X = df_processed.drop("income", axis=1)
y = df_processed["income"]

# Convertir las etiquetas categóricas a valores numéricos
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Definir las columnas categóricas para One-Hot Encoding
categorical_cols = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']

# Crear un transformador para aplicar One-Hot Encoding a las columnas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse=False), categorical_cols)
    ],
    remainder='passthrough'
)

# Crear un pipeline con el preprocesador y el escalador
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler(with_mean=False))  # Ajustar el parámetro with_mean
])

# Aplicar el pipeline al conjunto de entrenamiento y prueba
X_train_scaled = pipeline.fit_transform(X_train)
X_test_scaled = pipeline.transform(X_test)

# Crear modelo de red neuronal
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo con NeptuneCallback
model.fit(
    X_train_scaled,
    y_train_encoded,
    epochs=10,
    batch_size=32,
    validation_data=(X_test_scaled, y_test_encoded),
    callbacks=[NeptuneCallback(run=run)]
)

# Realizar predicciones en el conjunto de prueba
y_probabilities = model.predict(X_test_scaled)

# Obtener las clases predichas (aplicando un umbral de 0.5)
y_pred_dl = (y_probabilities > 0.5).astype(int)

# Convertir las predicciones a etiquetas
y_pred_dl = y_pred_dl.flatten()

# Convertir las etiquetas codificadas de vuelta a las etiquetas originales
y_test_original = label_encoder.inverse_transform(y_test_encoded)
y_pred_original = label_encoder.inverse_transform(y_pred_dl)

# Evaluar el rendimiento del modelo
accuracy_dl = accuracy_score(y_test_encoded, y_pred_dl)
classification_rep_dl = classification_report(y_test_encoded, y_pred_dl)
conf_matrix_dl = confusion_matrix(y_test_encoded, y_pred_dl)

# Registrar métricas en Neptune
run["accuracy"] = accuracy_dl
run["classification_report"] = classification_rep_dl
run["confusion_matrix"] = str(conf_matrix_dl)

# Finalizar la corrida
run.stop()

AttributeError: module 'neptune.new' has no attribute 'init'